In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime
from datetime import timedelta
from datetime import datetime
import tensorflow as tf
%matplotlib inline
%config IPCompleter.greedy=True
#data = pd.read_csv("D:\\DataSets\\Rawflight.csv")
data = pd.read_csv("data\\airindia_clean.csv")

In [2]:
def rel(data):
    corel = data.corr().index
    plt.figure(figsize = (10,10))
    corell=sns.heatmap(data[corel].corr(),annot = True,cmap = 'RdYlGn')

In [3]:
mins =[]
for t in flight.duration:
    m = (int(t.split()[0].split('h')[0])*60) + (int(t.split()[1].split('m')[0]))
    mins.append(m)
flight.replace(["BOM","HYD","NAG","PNQ"],[1,2,3,4],inplace = True)
flight['Duration'] = mins
flight['Departure_Date'] = pd.to_datetime(flight.Departure_Date,dayfirst =True)
flight['Search_Date'] = pd.to_datetime(flight.Search_Date,dayfirst =True)
flight['Day_Name'] = flight.Departure_Date.dt.weekday
flight['Dept_Month'] = flight.Departure_Date.dt.month
flight['Search_Month'] = flight.Search_Date.dt.month
flight['Diff_Month'] = flight.Departure_Date.dt.month - flight.Search_Date.dt.month
flight.drop_duplicates(inplace = True)
flight.drop('Unnamed: 0',axis = 1,inplace =True)
tofday = []
for i in flight['deptime']:
    j=int(i.split(':')[0])
    if(j >=0 and j<6):
        tofday.append(1)
    elif(j >=6 and j<12):
        tofday.append(2)
    elif(j>=12 and j< 18):
        tofday.append(3)
    else:
        tofday.append(4)
flight['deptime'] = tofday
tofday1 = []
for i in flight['arrtime']:
    j=int(i.split(':')[0])
    if(j >=0 and j<6):
        tofday1.append(1)
    elif(j >=6 and j<12):
        tofday1.append(2)
    elif(j>=12 and j< 18):
        tofday1.append(3)
    else:
        tofday1.append(4)
flight['arrtime'] = tofday1
data = flight.loc[flight['airline'] == 'Air India']
data.reset_index(drop = True,inplace = True)
data.head()

,Search_Date,Departure_Date,Days_fly,origin,destination,deptime,arrtime,flightno,airline,Total_fare,...,seatingclass,duration,stops,stop1,stop2,Duration,Day_Name,Dept_Month,Search_Month,Diff_Month
0,2019-08-07,2019-08-07,0,DEL,1,3,1,10,Air India,5771,...,E,12h 0m,1,AMD,NaN,720,2,8,8,0
1,2019-08-07,2019-08-08,1,DEL,1,3,2,10,Air India,5771,...,E,13h 20m,1,AMD,NaN,800,3,8,8,0
2,2019-08-07,2019-08-09,2,DEL,1,3,1,10,Air India,6821,...,E,12h 0m,1,AMD,NaN,720,4,8,8,0
3,2019-08-07,2019-08-09,2,DEL,1,3,2,10,Air India,7451,...,E,15h 55m,1,AMD,NaN,955,4,8,8,0
4,2019-08-07,2019-08-10,3,DEL,1,3,1,10,Air India,5666,...,E,12h 0m,1,AMD,NaN,720,5,8,8,0


In [4]:
data.drop(columns = {"origin","seatingclass","stop1","stop2","flightno","airline","seats_avail","duration"},axis = 1,inplace = True)
data['Search_Day'] = data.Search_Date.dt.day
fare_cat = []
for i in data['Total_fare']:
    
    if(i<=3000):
        fare_cat.append(0)
    elif(i>3000 and i<=4000):
        fare_cat.append(1)
    elif(i>4000 and i< 5000):
        fare_cat.append(2)
    elif(i>5000 and i<=6000):
        fare_cat.append(3)
    elif(i>6000 and i<=7000):
        fare_cat.append(4)
    elif(i>7000 and i <= 8000):
        fare_cat.append(5)
    else:
        fare_cat.append(6)
data['Fare_Cat'] = fare_cat
data['Diff_Month'] = [x if x>=0 else x+12 for x in data['Diff_Month']]
data.tail()
data.to_csv("AirIndia_clean.csv")

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [ ]:
data.shape

In [4]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
data.columns
X = data[['Days_fly','destination','deptime','stops','Day_Name','Diff_Month','Search_Day']]
Y = data['Fare_Cat']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(7,)),
    tf.keras.layers.Dense(200,activation = 'relu'),
    tf.keras.layers.Dense(150,activation = 'relu'),
    tf.keras.layers.Dense(100,activation = 'relu'),
    tf.keras.layers.Dense(50,activation = 'relu'),
    tf.keras.layers.Dense(25,activation = 'relu'),
    tf.keras.layers.Dense(7,activation = 'softmax'),
    
])

model.compile(optimizer = tf.keras.optimizers.Adam(),loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
model.fit(X_train,y_train,epochs =200)

Epoch 1/200
26469/26469 [==============================] - 50s 2ms/step - loss: 1.2502 - accuracy: 0.4506
Epoch 2/200
26469/26469 [==============================] - 53s 2ms/step - loss: 1.1838 - accuracy: 0.4766
Epoch 3/200
26469/26469 [==============================] - 47s 2ms/step - loss: 1.1613 - accuracy: 0.4863
Epoch 4/200
26469/26469 [==============================] - 44s 2ms/step - loss: 1.1460 - accuracy: 0.4922
Epoch 5/200
26469/26469 [==============================] - 47s 2ms/step - loss: 1.1262 - accuracy: 0.5030
Epoch 6/200
26469/26469 [==============================] - 44s 2ms/step - loss: 1.0948 - accuracy: 0.5199
Epoch 7/200
26469/26469 [==============================] - 44s 2ms/step - loss: 1.0597 - accuracy: 0.5382
Epoch 8/200
26469/26469 [==============================] - 48s 2ms/step - loss: 1.0292 - accuracy: 0.5517
Epoch 9/200
26469/26469 [==============================] - 39s 1ms/step - loss: 1.0051 - accuracy: 0.5632
Epoch 10/200
26469/26469 [====================

26469/26469 [==============================] - 46s 2ms/step - loss: 1.0748 - accuracy: 0.5262
Epoch 78/200
26469/26469 [==============================] - 44s 2ms/step - loss: 1.0742 - accuracy: 0.5271
Epoch 79/200
26469/26469 [==============================] - 45s 2ms/step - loss: 1.1039 - accuracy: 0.5142
Epoch 80/200
26469/26469 [==============================] - 40s 1ms/step - loss: 1.1396 - accuracy: 0.4943
Epoch 81/200
26469/26469 [==============================] - 39s 1ms/step - loss: 1.1275 - accuracy: 0.4989
Epoch 82/200
26469/26469 [==============================] - 39s 1ms/step - loss: 1.1431 - accuracy: 0.4920
Epoch 83/200
26469/26469 [==============================] - 40s 2ms/step - loss: 1.1611 - accuracy: 0.4863
Epoch 84/200
26469/26469 [==============================] - 40s 2ms/step - loss: 1.1515 - accuracy: 0.4905
Epoch 85/200
26469/26469 [==============================] - 42s 2ms/step - loss: 1.1495 - accuracy: 0.4915
Epoch 86/200
26469/26469 [========================

26469/26469 [==============================] - 52s 2ms/step - loss: 1.0732 - accuracy: 0.5341
Epoch 154/200
26469/26469 [==============================] - 47s 2ms/step - loss: 1.0634 - accuracy: 0.5381
Epoch 155/200
26469/26469 [==============================] - 49s 2ms/step - loss: 1.0853 - accuracy: 0.5280
Epoch 156/200
26469/26469 [==============================] - 45s 2ms/step - loss: 1.0663 - accuracy: 0.5372
Epoch 157/200
26469/26469 [==============================] - 55s 2ms/step - loss: 1.0755 - accuracy: 0.5340
Epoch 158/200
26469/26469 [==============================] - 53s 2ms/step - loss: 1.0636 - accuracy: 0.5383
Epoch 159/200
26469/26469 [==============================] - 53s 2ms/step - loss: 1.0629 - accuracy: 0.5398
Epoch 160/200
26469/26469 [==============================] - 48s 2ms/step - loss: 1.0861 - accuracy: 0.5292
Epoch 161/200
26469/26469 [==============================] - 49s 2ms/step - loss: 1.0770 - accuracy: 0.5317
Epoch 162/200
26469/26469 [===============

In [5]:
print(model.evaluate(X_test,y_test))
model.save("airindia.h5")

6618/6618 [==============================] - 8s 1ms/step - loss: 1.0955 - accuracy: 0.5280
[1.0954551696777344, 0.5280491709709167]
